In [1]:
#TO DO
#did not normalize or standardize
#nans in test set to 180

# Do I want to make the padding value 0 so i ignore all rows with 0??
# should i use a different optimizer for my compiler?? maybe rmsprop

In [134]:
import pandas as pd
import os
import csv
import numpy as np
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [135]:
ROOT_DATA_DIR = '../raw_data/Out_Feature_CSVs'
DATA_SUB_DIRs = ["Train", "Test"]
CLS_LIST = ["Bad", "Good"]

def get_y(data_path, class_list):
    
    data_dict = {"csv_paths":[], "csv_files":[], "labels":[]}
    
    for i, clss in enumerate(class_list):
        dir_path = os.path.join(data_path, clss)
        list_csvs = [l for l in os.listdir(dir_path) if l.split(".")[-1]=="csv"]
        print(f"Found {len(list_csvs)} CSVs in {dir_path} Directory")
        data_dict["csv_files"].extend(list_csvs)
        data_dict["labels"].extend([i for k in range(len(list_csvs))])
        data_dict["csv_paths"].extend([dir_path for _ in range(len(list_csvs))])
    
    total_csvs = len(data_dict["csv_files"])
    total_labels = len(data_dict["labels"])
    print(f"Number of csvs : {total_csvs}")
    print(f"Number of Labels : {total_labels}")
    return data_dict

train_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[0]),
                               class_list=CLS_LIST))


test_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[1]),
                               class_list=CLS_LIST))  

Found 40 CSVs in ../raw_data/Out_Feature_CSVs/Train/Bad Directory
Found 40 CSVs in ../raw_data/Out_Feature_CSVs/Train/Good Directory
Number of csvs : 80
Number of Labels : 80
Found 10 CSVs in ../raw_data/Out_Feature_CSVs/Test/Bad Directory
Found 10 CSVs in ../raw_data/Out_Feature_CSVs/Test/Good Directory
Number of csvs : 20
Number of Labels : 20


In [136]:
for i in train_data:
    print(train_data)

                                  csv_paths csv_files  labels
0    ../raw_data/Out_Feature_CSVs/Train/Bad    68.csv       0
1    ../raw_data/Out_Feature_CSVs/Train/Bad    54.csv       0
2    ../raw_data/Out_Feature_CSVs/Train/Bad    41.csv       0
3    ../raw_data/Out_Feature_CSVs/Train/Bad    55.csv       0
4    ../raw_data/Out_Feature_CSVs/Train/Bad    69.csv       0
..                                      ...       ...     ...
75  ../raw_data/Out_Feature_CSVs/Train/Good    11.csv       1
76  ../raw_data/Out_Feature_CSVs/Train/Good    13.csv       1
77  ../raw_data/Out_Feature_CSVs/Train/Good    12.csv       1
78  ../raw_data/Out_Feature_CSVs/Train/Good     9.csv       1
79  ../raw_data/Out_Feature_CSVs/Train/Good     8.csv       1

[80 rows x 3 columns]
                                  csv_paths csv_files  labels
0    ../raw_data/Out_Feature_CSVs/Train/Bad    68.csv       0
1    ../raw_data/Out_Feature_CSVs/Train/Bad    54.csv       0
2    ../raw_data/Out_Feature_CSVs/Train/Bad    

In [137]:
train_data.head()

,csv_paths,csv_files,labels
0,../raw_data/Out_Feature_CSVs/Train/Bad,68.csv,0
1,../raw_data/Out_Feature_CSVs/Train/Bad,54.csv,0
2,../raw_data/Out_Feature_CSVs/Train/Bad,41.csv,0
3,../raw_data/Out_Feature_CSVs/Train/Bad,55.csv,0
4,../raw_data/Out_Feature_CSVs/Train/Bad,69.csv,0


In [138]:
train_data['csv_paths'][0]

'../raw_data/Out_Feature_CSVs/Train/Bad'

In [139]:
train_data.tail()

,csv_paths,csv_files,labels
75,../raw_data/Out_Feature_CSVs/Train/Good,11.csv,1
76,../raw_data/Out_Feature_CSVs/Train/Good,13.csv,1
77,../raw_data/Out_Feature_CSVs/Train/Good,12.csv,1
78,../raw_data/Out_Feature_CSVs/Train/Good,9.csv,1
79,../raw_data/Out_Feature_CSVs/Train/Good,8.csv,1


In [140]:
y_train = train_data['labels']

In [141]:
y_test = test_data['labels']

In [142]:
y_test.shape
y_train.shape

(80,)

In [143]:
def csv_to_list_of_lists(csv_path):
    df = pd.read_csv(csv_path)
    df.fillna(0, inplace=True)
    list_of_lists = df.iloc[:,1:].values.tolist()
    return list_of_lists

In [144]:
def get_x(dataframe):
  list_of_csv_files = [os.path.join(row['csv_paths'], row["csv_files"]) for index, row in dataframe.iterrows()]
  X_list = [csv_to_list_of_lists(f) for f in list_of_csv_files]
  return X_list

In [145]:
#X_train is a list of lists of different list lengths
X_train = get_x(train_data)

In [146]:
#pad X_train so all arrays are of the same shape
X_train_pad = pad_sequences(X_train, dtype='float32',padding='post',value=-1000)
X_train_pad.shape

(80, 176, 2048)

In [147]:
#X_test is a list of lists of different list lengths
X_test = get_x(test_data)

In [148]:
#pad X_test so all arrays are of the same shape
X_test_pad = pad_sequences(X_test, dtype='float32',padding='post',value=-1000)
X_test_pad.shape

(20, 135, 2048)

In [185]:
print(X_train_pad.shape)
print(y_train.shape)
print(X_test_pad.shape)
print(y_test.shape)

(80, 176, 2048)
(80,)
(20, 135, 2048)
(20,)


In [162]:
X_train_pad = np.array(X_train_pad)
y_train = np.array(y_train)
X_test_pad = np.array(X_test_pad)
y_test =  np.array(y_test)

#LSTM RNN Model_1 (.55 - .60 accuracy)
model_LSTM = Sequential()

##change masking layer to padded amount
model_LSTM.add(layers.Masking(mask_value=-1000))

#how many units should the first layer be??
model_LSTM.add(layers.LSTM(20, activation='tanh'))  
model_LSTM.add(layers.Dense(10, activation='relu'))
model_LSTM.add(layers.Dense(1, activation='sigmoid'))

model_LSTM.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [198]:
#LSTM RNN Model_2 (.70 - accuracy)
model_LSTM = Sequential()

##change masking layer to padded amount
model_LSTM.add(layers.Masking(mask_value=-1000))

#how many units should the first layer be??
model_LSTM.add(layers.LSTM(200, activation='tanh'))
model_LSTM.add(layers.Dense(100, activation='relu'))
model_LSTM.add(layers.Dense(50, activation='relu'))
model_LSTM.add(layers.Dense(10, activation='relu'))
model_LSTM.add(layers.Dense(1, activation='sigmoid'))
model_LSTM.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics=['accuracy'])

In [199]:
#Fitting the model to the train set
es = EarlyStopping(patience=5)

model_LSTM.fit(X_train_pad, y_train, 
              epochs=5, 
              batch_size=32, 
              verbose=1, 
              callbacks = [es],
              shuffle=True ,
              validation_split=0.2)

Epoch 1/5
2/2 [==============================] - 8s 3s/step - loss: 0.6755 - accuracy: 0.6354 - val_loss: 1.0244 - val_accuracy: 0.1875
Epoch 2/5
2/2 [==============================] - 2s 876ms/step - loss: 0.6679 - accuracy: 0.6979 - val_loss: 0.5997 - val_accuracy: 0.4375
Epoch 3/5
2/2 [==============================] - 2s 977ms/step - loss: 0.6332 - accuracy: 0.6354 - val_loss: 1.2355 - val_accuracy: 0.1875
Epoch 4/5
2/2 [==============================] - 2s 974ms/step - loss: 0.5647 - accuracy: 0.6979 - val_loss: 0.5577 - val_accuracy: 0.8750
Epoch 5/5
2/2 [==============================] - 2s 1s/step - loss: 0.6413 - accuracy: 0.5938 - val_loss: 0.7671 - val_accuracy: 0.1875


##Hogan's first layer (do i need to add an input dimension??)
model.add(layers.SimpleRNN(500, input_dim=1, activation='relu'))

#how many units should the first layer be??
model.add(layers.LSTM(units=500, activation='tanh'))  
model.add(layers.Dense(250, activation='relu'))
model.add(layers.Dense(125, activation='relu'))
model.add(layers.Dense(75, activation='relu'))
model.add(layers.Dense(25, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [200]:
#Evaluate the model on the train set
model_LSTM.evaluate(X_train_pad, y_train)

3/3 [==============================] - 1s 234ms/step - loss: 0.6087 - accuracy: 0.6125


[0.6087079644203186, 0.612500011920929]

In [201]:
#Evaluate the model on the test set
model_LSTM.evaluate(X_test_pad, y_test)

1/1 [==============================] - 2s 2s/step - loss: 0.6094 - accuracy: 0.7000


[0.6094247698783875, 0.699999988079071]

#how many units should the first layer be??
model_LSTM.add(layers.LSTM(200, activation='tanh'))
model_LSTM.add(layers.Dense(100, activation='relu'))
model_LSTM.add(layers.Dense(50, activation='relu'))
model_LSTM.add(layers.Dense(25, activation='relu'))
model_LSTM.add(layers.Dense(5, activation='relu'))
model_LSTM.add(layers.Dense(1, activation='sigmoid'))


SECOND TRY
----------
train_high -  loss: 0.6067 - accuracy: 0.6979 - val_loss: 0.9396 - val_accuracy: 0.3125
eval - loss: 0.4998 - accuracy: 0.7000
        

THIRD TRY
----------
train_high -  loss: 0.6067 - accuracy: 0.6979 - val_loss: 0.9396 - val_accuracy: 0.3125
eval - loss: 0.4998 - accuracy: 0.7000

-----------------------------------------------------------------------------
model_LSTM.add(layers.LSTM(200, activation='tanh'))
model_LSTM.add(layers.Dense(150, activation='relu'))
model_LSTM.add(layers.Dense(75, activation='relu'))
model_LSTM.add(layers.Dense(20, activation='relu'))
model_LSTM.add(layers.Dense(1, activation='sigmoid'))
model_LSTM.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics=['accuracy'])

1st
-----
Test - 1/1 [==============================] - 2s 2s/step - loss: 0.6118 - accuracy: 0.6500
Median Train - 2/2 [==============================] - 2s 851ms/step - loss: 0.6207 - accuracy: 0.6875 - val_loss: 0.7819 - val_accur)
2nd
-----
Test - 1/1 [==============================] - 2s 2s/step - loss: 0.7145 - accuracy: 0.6500
Median Train - 2/2 [==============================] - 2s 910ms/step - loss: 0.5711 - accuracy: 0.7083 - val_loss: 1.1240 - val_accuracy: 0.2500

4th
-----
Test - 1/1 [==============================] - 2s 2s/step - loss: 0.6118 - accuracy: 0.6500
Median Train - 2/2 [==============================] - 2s 851ms/step - loss: 0.6207 - accuracy: 0.6875 - val_loss: 0.7819 - val_accur)

5th
-----
Test - 1/1 [==============================] - 2s 2s/step - loss: 0.7145 - accuracy: 0.6500
Median Train - 2/2 [==============================] - 2s 910ms/step - loss: 0.5711 - accuracy: 0.7083 - val_loss: 1.1240 - val_accuracy: 0.2500